In [ ]:
from plotnine import ggplot, aes, geom_point, labs, facet_wrap, geom_vline, theme_minimal, theme, element_text, geom_line, scale_x_continuous, ggtitle, xlab
import pandas as pd
import ast

In [ ]:
for dataset_name in ['detailed', 'systematic']:

    figures_path = 'figures/' + dataset_name + "/"
    acc_rate_csv_path = 'acc_rate/' + dataset_name + '.csv'

    # raw_df
    raw_df = pd.read_csv(acc_rate_csv_path)

    # create new columns named n_features
    new_col = []
    for i in range(raw_df.shape[0]):
        new_col.append(len(ast.literal_eval(raw_df['features'].to_numpy()[i])) + 1)
    df_n_features = pd.DataFrame(new_col, columns=['n_features'])

    # full df
    df = pd.concat([raw_df, df_n_features], axis=1)


    for n_layer in [1, 2, 3]:
        for n_features in [2, 3, 5]:
            df_fold = df
            df_fold_compare = df_fold[df_fold['n_layer'] == n_layer]
            df_fold_compare = df_fold_compare[df_fold_compare['n_features'] == n_features]
            df_fold_compare = df_fold_compare[df_fold_compare['f_engineer'] == 1]
            df_fold_compare = df_fold_compare[['fold', 'layer_size', 'acc']]

            plot = (
                ggplot(df_fold_compare, aes(x=pd.Categorical(df_fold_compare['layer_size']), y='acc')) +
                geom_line(aes(group=1)) +
                labs(title="dataset: " + dataset_name + " --- " + str(n_layer) + " hidden layer --- " + str(n_features) + " features",
                    x="hidden layer size",
                    y="acc percentage") +
                theme_minimal() + 
                facet_wrap('~fold', ncol=3) + 
                theme(plot_title=element_text(size=10))
            )

            # Display the plot
            plot.save(figures_path + "mlp_" + str(n_layer) + "_layers_" + str(n_features-1) + "_features.pdf", width=8, height=6)
            plot.save(figures_path + "mlp_" + str(n_layer) + "_layers_" + str(n_features-1) + "_features.jpg", width=8, height=6)